# ***Landsat ETM+ to OLI Harmonization***

[Fonte]('https://openprairie.sdstate.edu/cgi/viewcontent.cgi?referer=https://scholar.google.com/&httpsredir=1&article=1035&context=gsce_pubs')

[Referência]('https://developers.google.com/earth-engine/tutorials/community/landsat-etm-to-oli-harmonization')

Este tutorial trata da harmonização da refletância da superfície do Landsat ETM+ com a refletância da superfície do Landsat OLI. Ele fornece:

* Uma função de transformação espectral
* Funções para criar dados prontos para análise
* Um exemplo de visualização de série temporal

Destina-se a ser um guia de ponta a ponta para harmonizar e visualizar uma série temporal regional de mais de 35 anos de dados Landsat que podem ser imediatamente aplicadas à(s) sua(s) região(ões) de interesse.

Observe que os coeficientes para transformar ETM+ em OLI também se aplicam a TM. Assim, neste tutorial, a referência ao ETM+ é sinônimo de TM.

In [1]:
!pip install geemap
!pip install earthengine-api
!pip install geopandas
!pip install rasterio

     |████████████████████████████████| 2.0 MB 5.0 MB/s 
     |████████████████████████████████| 3.0 MB 33.0 MB/s 
     |████████████████████████████████| 3.3 MB 20.5 MB/s 
     |████████████████████████████████| 1.3 MB 28.6 MB/s 
     |████████████████████████████████| 130 kB 37.2 MB/s 
     |████████████████████████████████| 95 kB 2.0 MB/s 
     |████████████████████████████████| 99 kB 4.3 MB/s 
     |████████████████████████████████| 98 kB 6.1 MB/s 
     |████████████████████████████████| 187 kB 18.0 MB/s 
     |████████████████████████████████| 8.7 MB 36.1 MB/s 
     |████████████████████████████████| 1.2 MB 35.2 MB/s 
     |████████████████████████████████| 46 kB 1.9 MB/s 
     |████████████████████████████████| 128 kB 35.5 MB/s 
     |████████████████████████████████| 428 kB 40.2 MB/s 
     |████████████████████████████████| 53 kB 1.5 MB/s 
     |████████████████████████████████| 342 kB 36.3 MB/s 
     |████████████████████████████████| 130 kB 41.5 MB/s 
     |███████████████████

     |████████████████████████████████| 1.0 MB 5.3 MB/s 
     |████████████████████████████████| 16.7 MB 224 kB/s 
     |████████████████████████████████| 6.3 MB 32.3 MB/s 
     |████████████████████████████████| 19.3 MB 1.2 MB/s 


In [2]:
import ee
import geemap
import rasterio as rio
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy
import altair as alt

In [3]:
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=g9L5yut6eBiNaQaBFzWbvMqa_1mCGz_XERhcb-Gg8-8&tc=94FElqMuYyiLgakXJPqxKNXraI24hm9jqfALYPHdzBA&cc=LN7M8GgorN_G9N5sYGlUkjvtOpu3mS0Bp-Oa6NRk6PI

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWjK_OBftoDqTfvwqdPZx952Zo9tKUo-w8EJFWPNqzi8ev5PH6IMJXQ

Successfully saved authorization token.


# ***Por que harmonização?***

Roy et ai. (2016) demonstram que existem diferenças pequenas, mas potencialmente significativas, entre as características espectrais do Landsat ETM+ e OLI, dependendo da aplicação. As razões pelas quais você pode querer harmonizar conjuntos de dados incluem: produzir uma série temporal longa que abrange Landsat TM, ETM+ e OLI, gerar composições intra-anuais de data próxima para reduzir os efeitos de observações ausentes de gaps ETM+ SLC-off e mascaramento de nuvem/sombra , ou aumente a frequência de observação dentro de uma série temporal. Por favor, veja o manuscrito vinculado acima para mais informações.

# ***Harmonização***

A harmonização é conseguida através da transformação linear do espaço espectral ETM+ para o espaço espectral OLI de acordo com os coeficientes apresentados em Roy et al. (2016) Tabela 2 Coeficientes de regressão OLS. Os coeficientes relativos à banda são definidos no dicionário a seguir com as constantes de imagem de inclinação ( slopes) e interceptação ( ). itcpsObserve que os valores de interceptação y são multiplicados por 10.000 para corresponder à escala dos dados de refletância de superfície do USGS Landsat.

In [297]:
coefficients = {
  'itcps': ee.Image.constant([0.0003, 0.0088, 0.0061, 0.0412, 0.0254, 0.0172])
             .multiply(10000),
  'slopes': ee.Image.constant([0.8474, 0.8483, 0.9047, 0.8462, 0.8937, 0.9071])
};

In [298]:
print(coefficients.get('slopes'))

ee.Image({
  "functionInvocationValue": {
    "functionName": "Image.constant",
    "arguments": {
      "value": {
        "constantValue": [
          0.8474,
          0.8483,
          0.9047,
          0.8462,
          0.8937,
          0.9071
        ]
      }
    }
  }
})


Os nomes das bandas ETM+ e OLI para a mesma janela de resposta espectral não são iguais e precisam ser padronizados. As funções a seguir selecionam apenas as bandas de refletância e a pixel_qabanda de cada conjunto de dados e as renomeiam de acordo com a faixa de comprimento de onda que representam.

In [299]:
##Função para obter e renomear bandas de interesse do OLI.
def renameOli(img):
  return img.select(['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'pixel_qa'],['Blue', 'Green', 'Red', 'NIR', 'SWIR1', 'SWIR2', 'pixel_qa'])
  
##Função para obter e renomear bandas de interesse do ETM+.
def renameEtm(img):
  return img.select(['B1', 'B2', 'B3', 'B4', 'B5', 'B7', 'pixel_qa'],['Blue', 'Green', 'Red', 'NIR', 'SWIR1', 'SWIR2', 'pixel_qa'])

Por fim, defina a função de transformação, que aplica o modelo linear aos dados ETM+, lança o tipo de dados conforme Int16a consistência com OLI e reanexa a pixel_qabanda para uso posterior em mascaramento de nuvem e sombra.

In [300]:
def etmToOli(img):
  return img.select(['Blue', 'Green', 'Red', 'NIR', 'SWIR1', 'SWIR2'])\
  .multiply(coefficients.get('slopes'))\
  .add(coefficients.get('itcps'))\
  .round()\
  .toShort()\
  .addBands(img.select('pixel_qa'))

# ***Mascaramento de nuvens e sombras***

Os dados prontos para análise devem ter nuvens e sombras de nuvens mascaradas. A função a seguir usa a banda CFmask ( Zhu et al., 2015 ) pixel_qaincluída em cada imagem de refletância de superfície do Landsat USGS para definir pixels identificados como nuvem e sombra de nuvem como nulos.

In [301]:
def fmask(img):
  cloudShadowBitMask = 1 << 3;
  cloudsBitMask = 1 << 5;
  qa = img.select('pixel_qa');
  mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0).And(qa.bitwiseAnd(cloudsBitMask).eq(0));
  return img.updateMask(mask)

# ***Cálculo do índice espectral***

O próximo aplicativo usa o índice espectral de taxa de queima normalizada (NBR) para representar a história espectral de um pixel florestal afetado por incêndios florestais. NBR é usado porque Cohen et al. (2018) descobriram que entre 13 índices/bandas espectrais, o NBR teve a maior relação sinal/ruído em relação à perturbação florestal (sinal) em todos os EUA. É calculado pela seguinte função.

In [302]:
def calcNbr(img):
  return img.normalizedDifference(['NIR', 'SWIR2']).rename('NBR')

# ***Combinar funções***

Defina uma função wrapper para cada conjunto de dados que consolide todas as funções acima por conveniência ao aplicá-las às suas respectivas coleções de imagens.

In [303]:
##Define a função para preparar as imagens OLI.
def prepOli(img):
  orig = img
  img = renameOli(img)
  img = fmask(img)
  img = calcNbr(img)
  return ee.Image(img.copyProperties(orig, orig.propertyNames()))

##Define a função para preparar imagens ETM+.
def prepEtm(img):
  orig = img
  img = renameEtm(img)
  img = fmask(img)
  img = etmToOli(img)
  img = calcNbr(img)
  return ee.Image(img.copyProperties(orig, orig.propertyNames()))

# ***Exemplo de série temporal***

In [304]:
##Defina a área de interess
aoi = ee.Geometry.Point([-121.70938, 45.43185]);

In [305]:
##Defina as coleções que irá utilizar
oliCol = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')
etmCol = ee.ImageCollection('LANDSAT/LE07/C01/T1_SR')
tmCol = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR')

In [306]:
print(oliCol.first().bandNames().getInfo())
print(etmCol.first().bandNames().getInfo())
print(tmCol.first().bandNames().getInfo())

['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B10', 'B11', 'sr_aerosol', 'pixel_qa', 'radsat_qa']
['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'sr_atmos_opacity', 'sr_cloud_qa', 'pixel_qa', 'radsat_qa']
['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'sr_atmos_opacity', 'sr_cloud_qa', 'pixel_qa', 'radsat_qa']


In [307]:
##Definir um filtro de coleção de imagens
##Defina um filtro para restringir as coleções de imagens pelo limite espacial 
##da área de interesse, estação de pico da fotossíntese e qualidade.
colFilter = ee.Filter.And(
    ee.Filter.bounds(aoi), ee.Filter.calendarRange(182, 244, 'day_of_year'),
    ee.Filter.lt('CLOUD_COVER', 50), ee.Filter.lt('GEOMETRIC_RMSE_MODEL', 10),
    ee.Filter.Or(
        ee.Filter.eq('IMAGE_QUALITY', 9),
        ee.Filter.eq('IMAGE_QUALITY_OLI', 9)));

In [308]:
##Preparar as coleções
##Filtre as coleções e prepare-as para mesclagem.
oliCol = oliCol.filter(colFilter).map(prepOli);
etmCol = etmCol.filter(colFilter).map(prepEtm);
tmCol = tmCol.filter(colFilter).map(prepEtm);

##Mesclar as coleções.
col = oliCol.merge(etmCol).merge(tmCol);
print('Banda da coleção',col.first().bandNames().getInfo())




Banda da coleção ['NBR']


In [309]:
##Total de imagens Utilizadas
print(col.size().getInfo())

517


# ***Faça um gráfico de série temporal exibindo todas as observações***

A harmonia entre sensores pode ser avaliada qualitativamente traçando todas as observações como um gráfico de dispersão onde o sensor é discernido pela cor. 

Mapeie uma função de redução de região sobre a coleção de imagens que calcula a mediana de todos os pixels que cruzam a AOI e define o resultado como uma propriedade da imagem.

In [310]:
 def reduce(img):
      obs = img.reduceRegion(**
                             {'geometry': aoi, 'reducer': ee.Reducer.median(), 'scale': 30, 'maxPixels':1e13})
      return img.set('NBR', obs.get('NBR'))

In [311]:
def date (image):
    return image.copyProperties(image, ["system:time_start"]).set('date', image.date().format('YYYY-MM-dd')).set({'millis': image.date().millis()})


In [312]:
allObs = col.map(date).map(reduce)
print('Propriedades', allObs.first().propertyNames().getInfo())

Propriedades ['date', 'SATELLITE', 'SOLAR_AZIMUTH_ANGLE', 'IMAGE_QUALITY_OLI', 'WRS_PATH', 'system:time_start', 'LANDSAT_ID', 'SENSING_TIME', 'ESPA_VERSION', 'SOLAR_ZENITH_ANGLE', 'system:version', 'WRS_ROW', 'GEOMETRIC_RMSE_MODEL_Y', 'CLOUD_COVER_LAND', 'LEVEL1_PRODUCTION_DATE', 'GEOMETRIC_RMSE_MODEL_X', 'system:asset_size', 'millis', 'GEOMETRIC_RMSE_MODEL', 'SR_APP_VERSION', 'PIXEL_QA_VERSION', 'system:index', 'NBR', 'IMAGE_QUALITY_TIRS', 'system:footprint', 'CLOUD_COVER', 'system:id', 'EARTH_SUN_DISTANCE', 'system:bands', 'system:band_names']


In [313]:
##Estabelecendo a lista dos dados
Lista = allObs.reduceColumns(ee.Reducer.toList(4), ['SATELLITE','date','millis','NBR']).values().get(0)

In [314]:
##Convertendo para dataframe
df_nbr = pd.DataFrame(Lista.getInfo(), columns=['SATELLITE','date','millis','NBR'])
df_nbr

,SATELLITE,date,millis,NBR
0,LANDSAT_8,2013-07-12,1373655111580,-0.232396
1,LANDSAT_8,2013-07-28,1375037511700,-0.212800
2,LANDSAT_8,2013-08-13,1376419912430,-0.215797
3,LANDSAT_8,2014-07-15,1405450184480,-0.157320
4,LANDSAT_8,2014-07-31,1406832590110,-0.116492
...,...,...,...,...
382,LANDSAT_5,2009-07-24,1248461102410,0.633199
383,LANDSAT_5,2010-07-27,1280256389530,0.599819
384,LANDSAT_5,2010-08-12,1281638783120,0.633975
385,LANDSAT_5,2011-07-30,1312051502040,0.601076


In [315]:
# Função para adicionar variáveis de data ao DataFrame.
def add_date_info(df):
  df['Timestamp'] = pd.to_datetime(df['millis'], unit='ms')
  df['Year'] = pd.DatetimeIndex(df['Timestamp']).year
  df['Month'] = pd.DatetimeIndex(df['Timestamp']).month
  df['Day'] = pd.DatetimeIndex(df['Timestamp']).day
  df['DOY'] = pd.DatetimeIndex(df['Timestamp']).dayofyear
  return df

In [316]:
##Adicionando as datas no Dataframe
df_nbr = add_date_info(df_nbr)
df_nbr.head(5)

,SATELLITE,date,millis,NBR,Timestamp,Year,Month,Day,DOY
0,LANDSAT_8,2013-07-12,1373655111580,-0.232396,2013-07-12 18:51:51.580,2013,7,12,193
1,LANDSAT_8,2013-07-28,1375037511700,-0.212800,2013-07-28 18:51:51.700,2013,7,28,209
2,LANDSAT_8,2013-08-13,1376419912430,-0.215797,2013-08-13 18:51:52.430,2013,8,13,225
3,LANDSAT_8,2014-07-15,1405450184480,-0.157320,2014-07-15 18:49:44.480,2014,7,15,196
4,LANDSAT_8,2014-07-31,1406832590110,-0.116492,2014-07-31 18:49:50.110,2014,7,31,212


In [317]:
##Gráfico Série Temporal
alt.Chart(df_nbr).mark_point().transform_fold(
    ['LANDSAT_5', 'LANDSAT_7', 'LANDSAT_8']).encode(
    x=alt.X('date', title='Data',axis=alt.Axis(labelAngle=-80)),
    y=alt.Y('NBR:Q', title='NBR', scale=alt.Scale(domain=[-0.5, 1])),color=alt.Color('SATELLITE',
                   scale=alt.Scale(
            domain=['LANDSAT_5', 'LANDSAT_7', 'LANDSAT_8'],
            range=['violet', 'green','blue']),title ='Legenda'),
    
    
    tooltip=[
             alt.Tooltip('date:N', title='Data'),
             alt.Tooltip('NBR:Q'),
             alt.Tooltip('SATELLITE')
             
             ]).properties(title='Série histórica NBR por ano',width=2000, height=600).interactive()

alt.Chart(...)

In [318]:
df_mean = df_nbr.groupby(['NBR','Year']).median().reset_index()
df_mean

,NBR,Year,millis,Month,Day,DOY
0,-0.247891,2013,1.377025e+12,8.0,20.0,232.0
1,-0.240260,2013,1.374260e+12,7.0,19.0,200.0
2,-0.238752,2013,1.373655e+12,7.0,12.0,193.0
3,-0.232396,2013,1.373655e+12,7.0,12.0,193.0
4,-0.225421,2013,1.376420e+12,8.0,13.0,225.0
...,...,...,...,...,...,...
382,0.699955,1986,5.248591e+11,8.0,19.0,231.0
383,0.700540,2000,9.672281e+11,8.0,25.0,238.0
384,0.732283,2007,1.187031e+12,8.0,13.0,225.0
385,0.733265,1998,9.042429e+11,8.0,27.0,239.0


In [319]:
##Gráfico Série Temporal
alt.Chart(df_mean).mark_line().encode(
    x=alt.X('Year', title='Data',axis=alt.Axis(labelAngle=-80)),
    y=alt.Y('NBR:Q', title='NBR', scale=alt.Scale(domain=[-0.5, 1])),
    
    
    tooltip=[
             alt.Tooltip('Year:N', title='Data'),
             alt.Tooltip('NBR:Q'),
             
             ]).properties(title='Série histórica NBR por ano',width=800, height=400).interactive()

alt.Chart(...)

############################################### ################
### FUNÇÕES DE TRANSFORMAÇÃO ALTERNATIVA ###
############################################### ################

Roy et al. (2016) A Tabela 2 fornece coeficientes de regressão OLS e RMA
para transformar a refletância de superfície ETM+ em refletância de superfície OLI e vice-versa. 
O tutorial acima demonstra apenas a transformação de ETM+ para OLI por OLS regressão. Abaixo estão as funções para todas as opções de transformação. Modifique o acima funções de wrapper `prepOLI` e `prepETM` conforme necessário para adicionar/remover/substituir funções de transformação. Além disso, use o dicionário `coeficientes` abaixo em vez do definido acima (o seguinte inclui todos os conjuntos de coeficientes).

In [320]:
##Define OLS and RMA surface regression coefficients.
def coefficients(image):
  etm2oli_ols: {
      'itcps': ee.Image.constant([0.0003, 0.0088, 0.0061, 0.0412, 0.0254, 0.0172]).multiply(10000),
      'slopes': ee.Image.constant([0.8474, 0.8483, 0.9047, 0.8462, 0.8937, 0.9071])
      }
def oli2etm_ols(image): 
  {
          'itcps': ee.Image.constant([0.0183, 0.0123, 0.0123, 0.0448, 0.0306, 0.0116]).multiply(10000),
          'slopes': ee.Image.constant([0.885, 0.9317, 0.9372, 0.8339, 0.8639, 0.9165])
          }
def rma(image): 
  {
      'itcps': ee.Image.constant([-0.0095, -0.0016, -0.0022, -0.0021, -0.0030, 0.0029]).multiply(10000),
      'slopes': ee.Image.constant([0.9785, 0.9542, 0.9825, 1.0073, 1.0171, 0.9949])
      }
              

##Define function to apply OLS ETM+ to OLI transformation.
def etm2oli_ols(img):
  return img.select(['Blue', 'Green', 'Red', 'NIR', 'SWIR1', 'SWIR2'])\
  .multiply(coefficients.etm2oli_ols.slopes)\
  .add(coefficients.etm2oli_ols.itcps)\
  .round()\
  .toShort()\
  .addBands(img.select('pixel_qa')\
  );


##Define function to apply OLS OLI to ETM+ transformation.
def oli2etm_ols(img):
  return ee.Image(
      img.select(['Blue', 'Green', 'Red', 'NIR', 'SWIR1', 'SWIR2'])\
      .multiply(coefficients.oli2etm_ols.slopes)\
      .add(coefficients.oli2etm_ols.itcps)\
      .round()\
      .toShort()\
      .addBands(img.select('pixel_qa'))\
      .copyProperties(img, ['system:time_start'])\
  )

##Define function to apply RMA OLI to ETM+ transformation.
def oli2etm_rma(img):
  return ee.Image(
    img.select(['Blue', 'Green', 'Red', 'NIR', 'SWIR1', 'SWIR2'])
      .subtract(coefficients.rma.itcps)
      .divide(coefficients.rma.slopes)
      .round()
      .toShort()
      .addBands(img.select('pixel_qa'))
      .copyProperties(img, ['system:time_start'])
  );


##Define function to apply RMA ETM+ to OLI transformation.
def etm2oli_rma(img):
  return ee.Image(
    img.select(['Blue', 'Green', 'Red', 'NIR', 'SWIR1', 'SWIR2'])
      .multiply(coefficients.rma.slopes)
      .add(coefficients.rma.itcps)
      .round()
      .toShort()
      .addBands(img.select('pixel_qa'))
      .copyProperties(img, ['system:time_start'])
  )